# Converter for the Financial Filings at the Sec

In [1]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2020q3.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2020q3"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

21164760
10678830
533952
799047
24168


In [2]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+-----------------------------------------------------------+-----------+-----------------------------------------+----------+--------+--------+-------------------+------------+
|    | adsh                 | tag                                                       | version   | coreg                                   |    ddate |   qtrs | uom    |             value |   footnote |
|----+----------------------+-----------------------------------------------------------+-----------+-----------------------------------------+----------+--------+--------+-------------------+------------|
|  0 | 0001661920-20-000055 | EntityCommonStockSharesOutstanding                        | dei/2018  | nan                                     | 20200630 |      0 | shares |       2.49449e+07 |        nan |
|  1 | 0001469709-20-000039 | EntityCommonStockSharesOutstanding                        | dei/2018  | nan                                     | 20200630 |      0 | shares |    

In [3]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------+--------+--------+---------+---------+------------------------------------+--------------+-----------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                | version      | plabel                                              |   negating |
|----+----------------------+----------+--------+--------+---------+---------+------------------------------------+--------------+-----------------------------------------------------+------------|
|  0 | 0000024545-20-000018 |        1 |      4 | CP     |       0 | H       | CA                                 | country/2017 | CANADA                                              |          0 |
|  1 | 0001552033-20-000073 |       32 |     23 | UN     |       0 | H       | CA                                 | country/2017 | CANADA                                              |          0 |
|  2 | 000

In [4]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+---------+---------------------------------------+-------+-------------+----------+------------------+------------+------------------------------------+-------------------------------------+-----------------+-------------+----------+------------------+------------+------------------------------------+-------------------------------------+--------------+-----------+-------------+--------------------------------------------+---------------+-------+--------+-------+--------+-------------+------+------+----------+-----------------------+-----------+----------+----------------------------------+---------+-----------------+
|    | adsh                 |     cik | name                                  |   sic | countryba   | stprba   | cityba           | zipba      | bas1                               | bas2                                | baph            | countryma   | stprma   | cityma           | zipma      | mas1                               | mas2         

In [5]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+---------------------------------------------------------------------------+--------------+----------+------------+------------------+--------+--------+-----------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | tag                                                                       | version      |   custom |   abstract | datatype         | iord   | crdr   | tlabel                                                                                  | doc                                   

In [6]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+
adsh         object
tag          object
version      object
coreg        object
ddate         int64
qtrs          int64
uom          object
value       float64
footnote     object
dtype: object


In [7]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                      0001661920-20-000055
tag         EntityCommonStockSharesOutstanding
version                               dei/2018
coreg                                      NaN
ddate                                 20200630
qtrs                                         0
uom                                     shares
value                               24944891.0
footnote                                   NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [8]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [9]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [10]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(submitted["period"]))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        #FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 419 Dtos size: 147 Bs: 68 Cf: 79 Ic: 0 Json size: 35kb.
file 1 of 14833 exportFiles/2020q3/0000006955-20-000039.json stored in 8730.851ms.
FilterDfNum size: 241 Dtos size: 134 Bs: 64 Cf: 70 Ic: 0 Json size: 32kb.
file 4 of 14833 exportFiles/2020q3/0000016918-20-000173.json stored in 5236.842ms.
FilterDfNum size: 435 Dtos size: 177 Bs: 117 Cf: 60 Ic: 0 Json size: 45kb.
file 13 of 14833 exportFiles/2020q3/0000832988-20-000176.json stored in 9082.337ms.
FilterDfNum size: 183 Dtos size: 118 Bs: 72 Cf: 46 Ic: 0 Json size: 36kb.
file 15 of 14833 exportFiles/2020q3/0000844856-20-000019.json stored in 4022.18ms.
FilterDfNum size: 272 Dtos size: 123 Bs: 68 Cf: 55 Ic: 0 Json size: 29kb.
file 30 of 14833 exportFiles/2020q3/0001050825-20-000120.json stored in 5851.241ms.
FilterDfNum size: 388 Dtos size: 130 Bs: 68 Cf: 62 Ic: 0 Json size: 31kb.
file 58 of 14833 exportFiles/2020q3/0001170010-20-000131.json stored in 8055.554ms.
FilterDfNum size: 520 Dtos size: 155 Bs: 76 Cf: 79 Ic:

FilterDfNum size: 228 Dtos size: 83 Bs: 40 Cf: 43 Ic: 0 Json size: 22kb.
file 370 of 14833 exportFiles/2020q3/0001213900-20-016652.json stored in 5450.443ms.
File exportFiles/2020q3/0001213900-20-016686.json has Symbol Series([], )
FilterDfNum size: 241 Dtos size: 138 Bs: 87 Cf: 51 Ic: 0 Json size: 35kb.
file 373 of 14833 exportFiles/2020q3/0001213900-20-016728.json stored in 6175.897ms.
FilterDfNum size: 195 Dtos size: 100 Bs: 65 Cf: 35 Ic: 0 Json size: 23kb.
file 374 of 14833 exportFiles/2020q3/0001213900-20-016729.json stored in 4724.149ms.
FilterDfNum size: 460 Dtos size: 131 Bs: 75 Cf: 56 Ic: 0 Json size: 39kb.
file 375 of 14833 exportFiles/2020q3/0001213900-20-016745.json stored in 11220.686ms.
FilterDfNum size: 131 Dtos size: 58 Bs: 41 Cf: 17 Ic: 0 Json size: 14kb.
file 376 of 14833 exportFiles/2020q3/0001213900-20-016749.json stored in 3506.384ms.
FilterDfNum size: 77 Dtos size: 44 Bs: 31 Cf: 13 Ic: 0 Json size: 10kb.
file 378 of 14833 exportFiles/2020q3/0001213900-20-016760.js

FilterDfNum size: 445 Dtos size: 188 Bs: 98 Cf: 90 Ic: 0 Json size: 59kb.
file 514 of 14833 exportFiles/2020q3/0001487198-20-000010.json stored in 11587.346ms.
FilterDfNum size: 394 Dtos size: 157 Bs: 96 Cf: 61 Ic: 0 Json size: 37kb.
file 515 of 14833 exportFiles/2020q3/0001493152-20-012693.json stored in 10044.207ms.
FilterDfNum size: 469 Dtos size: 120 Bs: 57 Cf: 63 Ic: 0 Json size: 46kb.
file 524 of 14833 exportFiles/2020q3/0001564590-20-031962.json stored in 11700.691ms.
FilterDfNum size: 157 Dtos size: 98 Bs: 62 Cf: 36 Ic: 0 Json size: 23kb.
file 526 of 14833 exportFiles/2020q3/0001575705-20-000126.json stored in 4139.362ms.
FilterDfNum size: 173 Dtos size: 74 Bs: 51 Cf: 23 Ic: 0 Json size: 18kb.
file 529 of 14833 exportFiles/2020q3/0001594062-20-000061.json stored in 4468.119ms.
FilterDfNum size: 265 Dtos size: 134 Bs: 88 Cf: 46 Ic: 0 Json size: 30kb.
file 532 of 14833 exportFiles/2020q3/0001654954-20-007405.json stored in 6750.481ms.
File exportFiles/2020q3/0001684506-20-000006.

FilterDfNum size: 224 Dtos size: 99 Bs: 70 Cf: 29 Ic: 0 Json size: 24kb.
file 696 of 14833 exportFiles/2020q3/0001493152-20-012940.json stored in 5564.137ms.
FilterDfNum size: 228 Dtos size: 104 Bs: 71 Cf: 33 Ic: 0 Json size: 25kb.
file 697 of 14833 exportFiles/2020q3/0001493152-20-012942.json stored in 5753.233ms.
FilterDfNum size: 216 Dtos size: 108 Bs: 73 Cf: 35 Ic: 0 Json size: 26kb.
file 698 of 14833 exportFiles/2020q3/0001493152-20-012984.json stored in 5727.308ms.
FilterDfNum size: 169 Dtos size: 96 Bs: 72 Cf: 24 Ic: 0 Json size: 24kb.
file 699 of 14833 exportFiles/2020q3/0001493152-20-012987.json stored in 4273.572ms.
FilterDfNum size: 223 Dtos size: 102 Bs: 72 Cf: 30 Ic: 0 Json size: 25kb.
file 700 of 14833 exportFiles/2020q3/0001493152-20-012990.json stored in 5625.964ms.
FilterDfNum size: 222 Dtos size: 105 Bs: 73 Cf: 32 Ic: 0 Json size: 26kb.
file 701 of 14833 exportFiles/2020q3/0001493152-20-012996.json stored in 5680.093ms.
FilterDfNum size: 319 Dtos size: 106 Bs: 63 Cf: 

FilterDfNum size: 331 Dtos size: 143 Bs: 101 Cf: 42 Ic: 0 Json size: 69kb.
file 837 of 14833 exportFiles/2020q3/0001121781-20-000054.json stored in 8174.53ms.
FilterDfNum size: 367 Dtos size: 118 Bs: 64 Cf: 54 Ic: 0 Json size: 28kb.
file 844 of 14833 exportFiles/2020q3/0001185185-20-000984.json stored in 9080.163ms.
FilterDfNum size: 229 Dtos size: 114 Bs: 64 Cf: 50 Ic: 0 Json size: 26kb.
file 845 of 14833 exportFiles/2020q3/0001185185-20-000992.json stored in 5738.975ms.
FilterDfNum size: 184 Dtos size: 77 Bs: 49 Cf: 28 Ic: 0 Json size: 18kb.
file 865 of 14833 exportFiles/2020q3/0001213900-20-017386.json stored in 4671.482ms.
File exportFiles/2020q3/0001469709-20-000053.json has Symbol Series([], )
FilterDfNum size: 130 Dtos size: 65 Bs: 44 Cf: 21 Ic: 0 Json size: 14kb.
file 875 of 14833 exportFiles/2020q3/0001477932-20-003899.json stored in 3419.904ms.
FilterDfNum size: 159 Dtos size: 82 Bs: 52 Cf: 30 Ic: 0 Json size: 20kb.
file 876 of 14833 exportFiles/2020q3/0001477932-20-003920.js

...

FilterDfNum size: 273 Dtos size: 126 Bs: 66 Cf: 60 Ic: 0 Json size: 30kb.
file 13631 of 14833 exportFiles/2020q3/0001052918-20-000235.json stored in 6553.343ms.
FilterDfNum size: 564 Dtos size: 130 Bs: 79 Cf: 51 Ic: 0 Json size: 35kb.
file 13638 of 14833 exportFiles/2020q3/0001104659-20-106325.json stored in 13436.424ms.
File exportFiles/2020q3/0001104659-20-106526.json has Symbol Series([], )
File exportFiles/2020q3/0001104659-20-106534.json has Symbol Series([], )
File exportFiles/2020q3/0001104659-20-106545.json has Symbol Series([], )
File exportFiles/2020q3/0001104659-20-106552.json has Symbol Series([], )
FilterDfNum size: 43 Dtos size: 22 Bs: 15 Cf: 7 Ic: 0 Json size: 6kb.
file 13650 of 14833 exportFiles/2020q3/0001140361-20-020884.json stored in 1260.101ms.
FilterDfNum size: 426 Dtos size: 138 Bs: 90 Cf: 48 Ic: 0 Json size: 34kb.
file 13664 of 14833 exportFiles/2020q3/0001193125-20-248866.json stored in 10459.617ms.
File exportFiles/2020q3/0001193125-20-249097.json has Symbol S

FilterDfNum size: 144 Dtos size: 74 Bs: 48 Cf: 26 Ic: 0 Json size: 17kb.
file 13926 of 14833 exportFiles/2020q3/0001161697-20-000403.json stored in 3653.99ms.
FilterDfNum size: 1057 Dtos size: 202 Bs: 90 Cf: 112 Ic: 0 Json size: 52kb.
file 13935 of 14833 exportFiles/2020q3/0001193125-20-250594.json stored in 25746.488ms.
FilterDfNum size: 385 Dtos size: 129 Bs: 60 Cf: 69 Ic: 0 Json size: 33kb.
file 13968 of 14833 exportFiles/2020q3/0001213900-20-027772.json stored in 9727.993ms.
FilterDfNum size: 109 Dtos size: 0 Bs: 0 Cf: 0 Ic: 0 Json size: 0kb.
file 13969 of 14833 exportFiles/2020q3/0001213900-20-027777.json stored in 2724.42ms.
FilterDfNum size: 534 Dtos size: 139 Bs: 74 Cf: 65 Ic: 0 Json size: 33kb.
file 13975 of 14833 exportFiles/2020q3/0001387131-20-008427.json stored in 13467.242ms.
FilterDfNum size: 73 Dtos size: 41 Bs: 27 Cf: 14 Ic: 0 Json size: 10kb.
file 13976 of 14833 exportFiles/2020q3/0001387131-20-008433.json stored in 2061.188ms.
FilterDfNum size: 512 Dtos size: 129 Bs:

FilterDfNum size: 96 Dtos size: 44 Bs: 28 Cf: 16 Ic: 0 Json size: 10kb.
file 14242 of 14833 exportFiles/2020q3/0001477932-20-005572.json stored in 2576.99ms.
FilterDfNum size: 425 Dtos size: 134 Bs: 70 Cf: 64 Ic: 0 Json size: 66kb.
file 14257 of 14833 exportFiles/2020q3/0001564590-20-044516.json stored in 10701.381ms.
FilterDfNum size: 372 Dtos size: 152 Bs: 88 Cf: 64 Ic: 0 Json size: 35kb.
file 14271 of 14833 exportFiles/2020q3/0001654954-20-010386.json stored in 10108.107ms.
FilterDfNum size: 87 Dtos size: 48 Bs: 28 Cf: 20 Ic: 0 Json size: 12kb.
file 14275 of 14833 exportFiles/2020q3/0001683168-20-003237.json stored in 2246.286ms.
FilterDfNum size: 71 Dtos size: 48 Bs: 28 Cf: 20 Ic: 0 Json size: 12kb.
file 14276 of 14833 exportFiles/2020q3/0001683168-20-003241.json stored in 1861.185ms.
FilterDfNum size: 69 Dtos size: 46 Bs: 28 Cf: 18 Ic: 0 Json size: 11kb.
file 14277 of 14833 exportFiles/2020q3/0001683168-20-003243.json stored in 2007.094ms.
FilterDfNum size: 85 Dtos size: 46 Bs: 28

FilterDfNum size: 394 Dtos size: 163 Bs: 68 Cf: 95 Ic: 0 Json size: 36kb.
file 14494 of 14833 exportFiles/2020q3/0001213900-20-028570.json stored in 10000.276ms.
FilterDfNum size: 260 Dtos size: 122 Bs: 84 Cf: 38 Ic: 0 Json size: 28kb.
file 14495 of 14833 exportFiles/2020q3/0001213900-20-028572.json stored in 6621.933ms.
FilterDfNum size: 512 Dtos size: 161 Bs: 78 Cf: 83 Ic: 0 Json size: 39kb.
file 14497 of 14833 exportFiles/2020q3/0001213900-20-028638.json stored in 12886.965ms.
FilterDfNum size: 421 Dtos size: 155 Bs: 73 Cf: 82 Ic: 0 Json size: 38kb.
file 14498 of 14833 exportFiles/2020q3/0001213900-20-028650.json stored in 10792.427ms.
File exportFiles/2020q3/0001213900-20-028669.json has Symbol Series([], )
FilterDfNum size: 257 Dtos size: 92 Bs: 53 Cf: 39 Ic: 0 Json size: 22kb.
file 14500 of 14833 exportFiles/2020q3/0001213900-20-028670.json stored in 6419.891ms.
File exportFiles/2020q3/0001213900-20-028714.json has Period: nan
FilterDfNum size: 251 Dtos size: 129 Bs: 71 Cf: 58 Ic

FilterDfNum size: 200 Dtos size: 90 Bs: 44 Cf: 46 Ic: 0 Json size: 21kb.
file 14779 of 14833 exportFiles/2020q3/0001213900-20-029239.json stored in 4882.474ms.
FilterDfNum size: 145 Dtos size: 75 Bs: 53 Cf: 22 Ic: 0 Json size: 18kb.
file 14780 of 14833 exportFiles/2020q3/0001213900-20-029246.json stored in 3593.119ms.
FilterDfNum size: 512 Dtos size: 161 Bs: 78 Cf: 83 Ic: 0 Json size: 39kb.
file 14783 of 14833 exportFiles/2020q3/0001213900-20-029256.json stored in 12012.613ms.
FilterDfNum size: 305 Dtos size: 100 Bs: 68 Cf: 32 Ic: 0 Json size: 23kb.
file 14784 of 14833 exportFiles/2020q3/0001214659-20-008204.json stored in 7710.144ms.
FilterDfNum size: 181 Dtos size: 112 Bs: 68 Cf: 44 Ic: 0 Json size: 26kb.
file 14788 of 14833 exportFiles/2020q3/0001387131-20-008662.json stored in 4577.326ms.
FilterDfNum size: 172 Dtos size: 74 Bs: 43 Cf: 31 Ic: 0 Json size: 24kb.
file 14793 of 14833 exportFiles/2020q3/0001477932-20-005677.json stored in 4209.081ms.
FilterDfNum size: 183 Dtos size: 90 